In [35]:
ls

T403_在庫推移のグラフ可視化.ipynb
欠品リスク.ipynb
設計値変更後_データ作成_滞留日数.ipynb
設計値変更前_データ作成_滞留日数.ipynb
設計値変更後_データ作成_滞留日数_各工程.ipynb
かんばん数計算.ipynb
設計値変更前と後の比較.ipynb


In [32]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [33]:
#ライブラリのimport
import pandas as pd
import numpy as np
%matplotlib inline
#from matplotlib import pyplot as pyp
import matplotlib.pyplot as plt

In [49]:
#データ準備
#手順
#0.所在管理LTと行数合わせる必要ある。作り方は所在管理LTのコピー
#1.発注日時がNULL消す
#2.シム消す
#3.回収日時が7月8月のものは消す
#4.〇~〇LTから～を消す
#5.別ファイルにコピー
pre_zaikodata = pd.read_csv('..//data//設計値変更後_LTデータ.csv',encoding='shift_jis')
#ひとまずダミーデータで。
#pre_zaikodata = pd.read_csv('..//data//pre_zaikodata_20230828.csv',encoding='shift_jis')
#pre_zaikodata = pd.read_csv('sample1.csv',encoding='shift_jis')
pre_zaikodata.head(300)

/anaconda/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (20,37,38,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,かんばんシリアル,伝票番号,拠点所番地,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,...,A,B,C,整備室,箱種,箱サイズ,箱重量,基準在庫日数,基準在庫枚数,手配区分
0,0,1Z23G00071003,XCCA200,10114,019120LC040,ｷｬｯﾌﾟｼｯﾋﾟﾝｸﾞ,300,石黒ゴム工業（株）,< NULL >,回収済,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1Z23G00071004,XCCA210,10114,019120LC040,ｷｬｯﾌﾟｼｯﾋﾟﾝｸﾞ,300,石黒ゴム工業（株）,< NULL >,回収済,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1Z23H00052825,XCEA790,30072,351710LC010,ｶﾊﾞｰﾛｰﾀ,8,新日工業（株）,本社工場,回収済,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1Z23H00052826,XCEA790,30072,351710LC010,ｶﾊﾞｰﾛｰﾀ,8,新日工業（株）,本社工場,回収済,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1Z23H00053070,XCEA790,30072,351710LC010,ｶﾊﾞｰﾛｰﾀ,8,新日工業（株）,本社工場,回収済,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,1Z23H00060788,XCEF290,40135,9036652A003,ﾚｰｽﾃｰﾊﾟｰﾄﾞﾛｰﾗｰﾍﾞｱﾘﾝｸﾞｲﾝﾅｰ,60,（株）ジェイテクト,香川工場,回収済,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,296,1Z23H00060789,XCEF290,40133,9036652A004,ﾚｰｽﾃｰﾊﾟｰﾄﾞﾛｰﾗｰﾍﾞｱﾘﾝｸﾞｱｳﾀｰ,63,（株）ジェイテクト,香川工場,回収済,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,297,1Z23H00060790,XCEF290,40131,9036658A002,ﾚｰｽﾃｰﾊﾟｰﾄﾞﾛｰﾗｰﾍﾞｱﾘﾝｸﾞｱｳﾀｰ,36,（株）ジェイテクト,香川工場,回収済,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
298,298,1Z23H00060791,XCEF290,40139,9036659A009,ﾚｰｽﾃｰﾊﾟｰﾄﾞﾛｰﾗｰﾍﾞｱﾘﾝｸﾞｲﾝﾅｰ,36,（株）ジェイテクト,香川工場,回収済,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
#確認用（実行は不要）
pre_zaikodata.isnull().sum()

NameError: name 'pre_zaikodata' is not defined

In [6]:
###LT

long = pre_zaikodata.iloc[:,1]
count = 0
for i in range(len(long)):
    if i % 5000==0:
            print(i)
    if ((pre_zaikodata.loc[i, '発注印刷LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注検収LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注順立装置入庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注順立装置出庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注回収LT'])!='< NULL >'):
    #if (not pd.isnull(pre_zaikodata.loc[i, '発注印刷LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注検収LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置入庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置出庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注回収LT'])):
        LT2 = float(pre_zaikodata.loc[i, '発注検収LT']) - float(pre_zaikodata.loc[i, '発注印刷LT'])
        LT3 = float(pre_zaikodata.loc[i, '発注順立装置入庫LT'])-float(pre_zaikodata.loc[i, '発注検収LT'])
        LT4 = float(pre_zaikodata.loc[i, '発注順立装置出庫LT'])-float(pre_zaikodata.loc[i, '発注順立装置入庫LT'])
        LT5 = float(pre_zaikodata.loc[i, '発注回収LT'])-float(pre_zaikodata.loc[i, '発注順立装置出庫LT'])
        LT6 = float(pre_zaikodata.loc[i, '発注回収LT'])-float(pre_zaikodata.loc[i, '発注検収LT'])
        pre_zaikodata.loc[i, '印刷検収LT'] = LT2
        pre_zaikodata.loc[i, '検収入庫LT'] = LT3
        pre_zaikodata.loc[i, '入庫出庫LT'] = LT4
        pre_zaikodata.loc[i, '出庫回収LT'] = LT5
        pre_zaikodata.loc[i, '社内LT'] = LT6
        count = count + 1
print(count)

0
5000
10000
15000
20000
25000
30000
35000
40000
40348


In [7]:
###基準在庫枚数加味する

#稼働日入力
#kadoubi=22

pre_zaikodata['回収日時'] = pd.to_datetime(pre_zaikodata['回収日時'], errors='coerce')

count = 0
tehaidata = pd.read_csv('..//data//設計値変更後_202309_手配数_1Y_2次.csv',encoding='cp932')
#tehaidata = pd.read_csv('sample3.csv',encoding='cp932')
long2 = tehaidata.iloc[:,1]

for i in range(len(long)):
    if ((pre_zaikodata.loc[i, '発注印刷LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注検収LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注順立装置入庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注順立装置出庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注回収LT'])!='< NULL >'):
    #if (not pd.isnull(pre_zaikodata.loc[i, '発注印刷LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注検収LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置入庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置出庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注回収LT'])):
        hinban = pre_zaikodata.loc[i,'品番'] 
        hinban2 = hinban.replace("-", "") 
        hinban3 = hinban2.replace(" ", "") 
        #print(i)
        if i % 5000==0:
                print(i)
        for j in range(len(long2)):
            #print(j)
            tehaihinban = tehaidata.loc[j,'品番'].replace('-', '')
            tehaihinban2 = tehaihinban.replace(' ', '')
            if hinban3 == tehaihinban2:
                day = pre_zaikodata.loc[i,'回収日時'].day
                index = tehaidata.columns.get_loc("翌々々月稼働日数") + day
                pre_zaikodata.loc[i,'日量数'] = tehaidata.iloc[j,index]
                if int(str(tehaidata.loc[j,'当月計算数']).replace(',', '')) != 0:
                    avenichiryo = int(str(tehaidata.loc[j,'当月計算数']).replace(',', ''))/int(tehaidata.loc[j,'当月稼働日数'])
                    pre_zaikodata.loc[i,'平均日量数'] = avenichiryo
                    pre_zaikodata.loc[i,'基準在庫枚数日数'] = (int(tehaidata.loc[j,'基準在庫枚数'])*pre_zaikodata.loc[i, '収容数'])/avenichiryo
                else:
                    pre_zaikodata.loc[i,'平均日量数'] = 0
                    pre_zaikodata.loc[i,'基準在庫枚数日数'] = 0
print(count)

0
5000
10000
15000
20000
30000
35000
0


In [8]:
count = 0
tehaidata = pd.read_csv('..//data//設計値変更後_XR10_手配運用情報2300911_1Y（回収引当適用）.csv',encoding='cp932')
#tehaidata = pd.read_csv('sample2.csv',encoding='cp932')
long2 = tehaidata.iloc[:,1]

for i in range(len(long)):
    if (not pd.isnull(pre_zaikodata.loc[i, '発注印刷LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注検収LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置入庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置出庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注回収LT'])):
        hinban = pre_zaikodata.loc[i,'品番'] 
        hinban2 = hinban.replace("-", "") 
        hinban3 = hinban2.replace(" ", "") 
        if i % 5000==0:
            print(i,hinban2)
        for j in range(len(long2)):
            tehaihinban = tehaidata.loc[j,'品番'].replace('-', '')
            tehaihinban2 = tehaihinban.replace(' ', '')
            #print(tehaihinban)
            if hinban3 == tehaihinban2:
                pre_zaikodata.loc[i, 'A'] = tehaidata.loc[j,'納入ｻｲｸﾙ間隔']
                pre_zaikodata.loc[i, 'B'] = tehaidata.loc[j,'納入ｻｲｸﾙ回数']
                pre_zaikodata.loc[i, 'C'] = tehaidata.loc[j,'納入ｻｲｸﾙ情報']
                pre_zaikodata.loc[i, '整備室'] = tehaidata.loc[j,'整備室']
                pre_zaikodata.loc[i, '箱種'] = tehaidata.loc[j,'登録箱種']
                pre_zaikodata.loc[i, '箱サイズ'] = tehaidata.loc[j,'容積(m3)']
                pre_zaikodata.loc[i, '箱重量'] = tehaidata.loc[j,'総重量(Kg)']
                #pre_zaikodata.loc[i, '不当ピッチ'] = tehaidata.loc[j,'不当ピッチ']
                #pre_zaikodata.loc[i, '通常品'] = tehaidata.loc[j,'通常品']
                #pre_zaikodata.loc[i, '選択品'] = tehaidata.loc[j,'選択品']
                pre_zaikodata.loc[i, '基準在庫日数'] = tehaidata.loc[j,'基準在庫日数']
                pre_zaikodata.loc[i, '基準在庫枚数'] = tehaidata.loc[j,'基準在庫枚数']
                pre_zaikodata.loc[i, '手配区分'] = tehaidata.loc[j,'手配区分']
                MAX= tehaidata.loc[j,'基準在庫日数'] + float(pre_zaikodata.loc[i,'基準在庫枚数日数'])
                pre_zaikodata.loc[i,'基準在庫日数（基準在庫枚数込み）'] = MAX
                zisseki = pre_zaikodata.loc[i, '社内LT'] 
                pre_zaikodata.loc[i, '実績在庫日数'] = zisseki
                if  (zisseki > MAX*0)and(zisseki < 0.25*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常1'
                elif  (zisseki > MAX*0.25)and(zisseki < 0.5*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常2'
                elif  (zisseki > MAX*0.5)and(zisseki < 0.75*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常3'
                elif  (zisseki > MAX*0.75)and(zisseki < MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常4'
                elif (zisseki >= MAX)and(zisseki < MAX*1.25):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常1'
                elif (zisseki >= MAX*1.25)and(zisseki < 1.5*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常2'
                elif (zisseki >= MAX*1.5)and(zisseki < 2*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常3'
                elif (zisseki >= MAX*2)and(zisseki < 3*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常4'
                elif zisseki >= 3*MAX:
                    pre_zaikodata.loc[i, '在庫状態'] = '異常5'
        #print(LT2,LT3,LT4,LT5)
        count = count + 1
print(count)

0 019120LC040    
5000 G9201ECE010    
10000 35771ECB010    
15000 9030119A011    
20000 9010512A018    
25000 39171ECB010    
30000 41310ECE010    
35000 G115A34010     
40000 35141ECE010    
44948


In [14]:
count = 0
sedata = pd.read_csv('..//data//設計値変更後_各工程の設計値.csv',encoding='cp932')
long2 = sedata.iloc[:,1]

for i in range(len(long)):
    if (not pd.isnull(pre_zaikodata.loc[i, '発注印刷LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注検収LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置入庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置出庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注回収LT'])):
        hinban = pre_zaikodata.loc[i,'品番'] 
        hinban2 = hinban.replace("-", "") 
        hinban3 = hinban2.replace(" ", "") 
        if i % 5000==0:
            print(i,hinban2)
        for j in range(len(long2)):
            sehinban = sedata.loc[j,'品番'].replace('-', '')
            sehinban2 = sehinban.replace(' ', '')
            #print(tehaihinban)
            if hinban3 == sehinban2:
                pre_zaikodata.loc[i, '設計値_検収入庫LT'] = sedata.loc[j,'検収入庫']
                pre_zaikodata.loc[i, '設計値_入庫出庫LT'] = sedata.loc[j,'入庫出庫']
                pre_zaikodata.loc[i, '設計値_出庫回収LT'] = sedata.loc[j,'出庫組立'] + sedata.loc[j,'引当回収']
                pre_zaikodata.loc[i, '設計値_出庫回収LT（基準在庫枚数込み）'] = sedata.loc[j,'出庫組立'] + sedata.loc[j,'引当回収'] + pre_zaikodata.loc[i,'基準在庫枚数日数']
                pre_zaikodata.loc[i, '差分_検収入庫LT'] = pre_zaikodata.loc[i, '検収入庫LT'] - sedata.loc[j,'検収入庫']
                pre_zaikodata.loc[i, '差分_入庫出庫LT'] = pre_zaikodata.loc[i, '入庫出庫LT'] - sedata.loc[j,'入庫出庫']
                pre_zaikodata.loc[i, '差分_出庫回収LT'] = pre_zaikodata.loc[i, '出庫回収LT'] - sedata.loc[j,'出庫組立'] - sedata.loc[j,'引当回収']
                pre_zaikodata.loc[i, '差分_出庫回収LT（基準在庫枚数込み）'] = pre_zaikodata.loc[i, '出庫回収LT'] - sedata.loc[j,'出庫組立'] - sedata.loc[j,'引当回収'] - pre_zaikodata.loc[i,'基準在庫枚数日数']

0 019120LC040    
5000 G9201ECE010    
10000 35771ECB010    
15000 9030119A011    
20000 9010512A018    
25000 39171ECB010    
30000 41310ECE010    
35000 G115A34010     
40000 35141ECE010    


In [ ]:
for i in range(len(long)):
    if (not pd.isnull(pre_zaikodata.loc[i, '発注印刷LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注検収LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置入庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置出庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注回収LT'])):
        hinban = pre_zaikodata.loc[i,'品番'] 
        hinban2 = hinban.replace("-", "") 
        hinban3 = hinban2.replace(" ", "") 
        if i % 5000==0:
            print(i,hinban2)
        for j in range(len(long2)):
            tehaihinban = tehaidata.loc[j,'品番'].replace('-', '')
            tehaihinban2 = tehaihinban.replace(' ', '')
            #print(tehaihinban)
            if hinban3 == tehaihinban2:
                zisseki = pre_zaikodata.loc[i, '検収入庫LT'] 
                MAX = pre_zaikodata.loc[i, '設計値_検収入庫LT']
                if  (zisseki > MAX*0)and(zisseki < 0.25*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常1'
                elif  (zisseki > MAX*0.25)and(zisseki < 0.5*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常2'
                elif  (zisseki > MAX*0.5)and(zisseki < 0.75*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常3'
                elif  (zisseki > MAX*0.75)and(zisseki < MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常4'
                elif (zisseki >= MAX)and(zisseki < MAX*1.25):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常1'
                elif (zisseki >= MAX*1.25)and(zisseki < 1.5*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常2'
                elif (zisseki >= MAX*1.5)and(zisseki < 2*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常3'
                elif (zisseki >= MAX*2)and(zisseki < 3*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常4'
                elif zisseki >= 3*MAX:
                    pre_zaikodata.loc[i, '在庫状態'] = '異常5'
        #print(LT2,LT3,LT4,LT5)
        count = count + 1
print(count)

In [25]:
with open('..//data//設計値変更後_LTデータ9月.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    pre_zaikodata.to_csv(f)

In [17]:
#指定日時を取り出す

pre_zaikodata['回収日時'] = pd.to_datetime(pre_zaikodata['回収日時'], errors='coerce')
pre_zaikodata['回収日時']    = pd.to_datetime(pre_zaikodata['回収日時'], errors='coerce')
pre_zaikodata['納入日'] = pd.to_datetime(pre_zaikodata['納入日'], errors='coerce')
pre_zaikodata['納入日']    = pd.to_datetime(pre_zaikodata['納入日'], errors='coerce')

#月から土曜の9時まで
pre_zaikodata2 = pre_zaikodata[((pre_zaikodata['納入日'] >= '2023-09-04') & 
                              (pre_zaikodata['回収日時'] <= '2023-09-08-09')) | ((pre_zaikodata['納入日'] >= '2023-09-11') & 
                              (pre_zaikodata['回収日時'] <= '2023-09-15-09')) | ((pre_zaikodata['納入日'] >= '2023-09-18') & 
                              (pre_zaikodata['回収日時'] <= '2023-09-22-09'))]
#田中さんから頂いた3月データはシムが含まれている。1Yでないものもある
print("設計値変更後：","元データのかんばん数",len(pre_zaikodata),"抽出したデータのかんばん数",len(pre_zaikodata2),"品番数",
      len(pre_zaikodata.loc[:,'品番'].unique()))

設計値変更後： 元データのかんばん数 44948 抽出したデータのかんばん数 26301 品番数 261


In [19]:
with open('..//data//設計値変更後_LTデータ9月（土日削除）.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    pre_zaikodata2.to_csv(f)

In [14]:
#pre_zaikodata.head(300)
tehaidata.head(300)

,品番,仕入先名/工場名,箱種類,収容数,基準在庫日数,基準在庫枚数,サイクル間隔,サイクル回数,サイクル情報,当月計算数,...,21(水),22(木),23(金),24(土),25(日),26(月),27(火),28(水),29(木),30(金)
0,35300-ECB010,アイシン精機（株）,TP-341 ﾊﾝﾖｳ,12,0.91,3,1,4,8.04,"21,214",...,946,"1,027",879,0,0,946,"1,029",945,"1,028",945
1,35580-ECB011,アイシン精機（株）,TP-342 ｾﾝﾖｳ,6,0.90,3,1,4,8.04,"9,254",...,426,456,397,0,0,411,445,411,452,405
2,1040 052 001Z,（株）青山製作所,TP-131 ﾊﾝﾖｳ,200,0.53,3,1,6,5.76,"11,959",...,520,571,482,0,0,535,584,534,576,540
3,1040 183 011P,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500,0.54,3,1,6,5.76,"27,762",...,"1,278","1,368","1,191",0,0,"1,233","1,335","1,233","1,356","1,215"
4,3040 052 001B,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500,0.52,3,1,6,5.76,"96,815",...,"4,304","4,679","3,998",0,0,"4,319","4,700","4,314","4,688","4,320"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,90564-57A157,（株）ムロコーポレーション,TP-131 ﾊﾝﾖｳ,100,0.93,3,1,1,0.73,0,...,0,0,0,0,0,0,0,0,0,0
296,90564-57A158,（株）ムロコーポレーション,TP-131 ﾊﾝﾖｳ,100,0.93,3,1,1,0.73,0,...,0,0,0,0,0,0,0,0,0,0
297,90564-57A159,（株）ムロコーポレーション,TP-131 ﾊﾝﾖｳ,100,0.93,3,1,1,0.73,0,...,0,0,0,0,0,0,0,0,0,0
298,90564-63A011,（株）ムロコーポレーション,TP-131 ﾊﾝﾖｳ,100,0.93,3,1,1,0.73,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
7/3

2.3333333333333335